# Colorado debt collector data

The goal: Extract data tables from [this PDF](https://coag.gov/sites/default/files/contentuploads/cp/ConsumerCreditUnit/InternetReports/carreport_0.pdf) and load into pandas for some basic analysis.

In [86]:
from os import path

import requests
import pdfplumber
import pandas as pd

### If PDF isn't there, download it

In [15]:
PDF_URL = 'https://coag.gov/sites/default/files/' \
          'contentuploads/cp/ConsumerCreditUnit/InternetReports/carreport_0.pdf'

TARGET_PDF = 'collections.pdf' 

if not path.isfile(TARGET_PDF):
    r = requests.get(PDF_URL)
    
    with open(TARGET_PDF, 'w') as f:
        for block in r.iter_content(1024):
            f.write(block)

### Define dataframe columns and status codes

In [45]:
cols = ['bizname', 'license_loc', 'instate_loc', 'mailing_loc',
        'license_no', 'lic_date', 'status', 'cr_date', 'action']

status_codes = {
    'A': 'active',
    'C': 'cancelled',
    'D': 'denied',
    'E': 'expired',
    'R': 'revoked'
}

### Extraction function
A function that takes a `pdfplumber.Page` object and returns a dataframe of data extracted from the table on that page.

In [17]:
def page_to_df(page):
    table = page.extract_tables()
    lines = table[0][1:]
    return pd.DataFrame(lines, columns=cols)

### Build the dataframe

In [37]:
# create an empty dataframe
df = pd.DataFrame(columns=cols)

# open the PDF
with pdfplumber.open(TARGET_PDF) as pdf:
    
    # skip the first page, which doesn't have a data table
    pages_with_data = pdf.pages[1:]
    
    # loop over the pages with data
    for page in pages_with_data:
        
        # call the extraction function on the page and append the result to our dataframe
        df = df.append(page_to_df(page), ignore_index=True)

In [82]:
# kill line breaks
df.replace('\n', ' ', inplace=True, regex=True)

In [84]:
# coerce license date col to datetime and sort descending
df.lic_date = pd.to_datetime(df.lic_date)
df = df.sort_values('lic_date', ascending=False)

In [68]:
df.head()

,bizname,license_loc,instate_loc,mailing_loc,license_no,lic_date,status,cr_date,action
329,BROAD STREET ASSET MANAGEMENT LLC,"2098 W CHESTER PIKE STE 201 BROOMALL, PA 19008","80 GARDEN CTR BLDG 3, STE B BROOMFIELD, CO 80020","2098 W CHESTER PIKE STE 201 BROOMALL, PA 19008",993088,2017-10-13,A,Active,
1142,HOME POINT FINANCIAL CORPORATION,"1194 OAK VALLEY DR STE 80 ANN ARBOR, MI 48108",3773 CHERRY CREEK DR N STE 575 OFFICE #39 DEN...,"1194 OAK VALLEY DR STE 80 ANN ARBOR, MI 48108",993089,2017-10-13,A,Active,
1088,GREAT LAKES PROCESSING SERVICES LLC,"1800 ELMWOOD AVE STE 2 BUFFALO, NY 14207","3025 S PARKER RD STE 705 AURORA, CO 80014","1800 ELMWOOD AVE STE 2 BUFFALO, NY 14207",993085,2017-10-11,A,Active,
2086,SUTHERLAND MORTGAGE SERVICES INC,"5959 CORPORATE DR STE 3000 HOUSTON, TX 77036","1776 S JACKSON ST STE 900 DENVER, CO 80210","5959 CORPORATE DR STE 3000 HOUSTON, TX 77036",993086,2017-10-11,A,Active,
575,COMMONWEALTH FINANCIAL SYSTEMS INC,"245 MAIN ST DICKSON CITY, PA 18519","80 GARCEN CTR BLDG B, STE3 BROOMFIELD, CO 80020","245 MAIN ST DICKSON CITY, PA 18519",993083,2017-10-10,A,Active,Yes


In [78]:
record_count = len(df)

### How much action?

According to the Colorado Attorney General, the presence of "Yes" in the last column means that the company has been

>subject to legal or administrative action by this office or the licensee entered into a voluntary settlement with this office. If the entry is "yes," the licensee may have been subject to one or more letters of admonition, suspension of the license, a judgment or order against the licensee, or other action, including payments (fines, penalties, consumer refunds, or other monetary payments.) Additionally, "yes" may mean that the licensee's records include a voluntary settlement or stipulation with this office. If a licensee has been disciplined, it might still retain its license. Actions and settlements are matters of public record although research, copying, and mailing costs may apply. Contact this office for more information.

What's that universe look like?

In [85]:
action = df[df.action == 'Yes']

print(len(action), 'of', '{:,}'.format(record_count))
print('{:.2f}% of debt collectors licensed in Colorado have been subject to some form of legal or administrative action.'.format((len(action) / record_count) * 100))

677 of 2,365
28.63% of debt collectors licensed in Colorado have been subject to some form of legal or administrative action.


In [ ]:
# tk: grouping by status code, analyze licenses by year, etc.